MPDATA import

In [1]:
from MPyDATA import ScalarField, VectorField, PeriodicBoundaryCondition
import numpy as np
import matplotlib.pyplot as plt

ImportError: cannot import name 'VectorField'

Auxiliary functions

           

In [ ]:
from numpy import sign
import numpy as np
from numpy import gradient


#extrapolation to value^{t+0.5}
def extrapolate_in_time(u_old, u_new):
    u_mid = (3. * u_new - u_old) * .5
    return u_mid


def grad(mass, dx=1.):
    return gradient(mass) / dx


def interpolate_in_space(uh, h):
    u = np.zeros((uh.shape[0] + 1))
    result = np.array(uh)
    
    zero_h = h==0
    non_zero_h = h!=0.
    result[non_zero_h] /= h[non_zero_h]
    result[zero_h] = 0.
    
    for i in range(1, u.shape[0] - 1):
        u[i] = (result[i-1] + result[i]) / 2.

    return u 

Advectee and advector initialization

In [ ]:
dx = 2.4
dt = 0.1

from MPyDATA.options import Options
options = Options(n_iters=2, infinite_gauge=True, flux_corrected_transport=True)

nx = 40

#initial state of liquid
initial_m_value = 0.2
h_initial = np.full(nx, initial_m_value)


#modification of liquid

h_initial[8:20] = np.array([0.8 - (0.02 * i)  for i in range(12)])

h_old = np.array(h_initial)
h_new = np.array(h_initial)

uh_new = np.zeros(nx)
uh_old = np.zeros(nx)

u_initial = np.zeros((nx + 1))

u_old = np.array(u_initial)
u_new = np.array(u_initial)

rhs_old = np.array(h_old)

halo = options.n_halo

advectee = ScalarField(
    data = h_new,
    halo=halo, 
    boundary_conditions=(PeriodicBoundaryCondition(), )
)

advector = VectorField(
    data = (u_initial,),
    halo=halo,
    boundary_conditions=(PeriodicBoundaryCondition(), )   
)

print('h_new', h_new,'\n')
print('u_new', u_initial,'\n')
print('grad', grad(h_new),'\n')

Stepper setup

In [ ]:
from MPyDATA.stepper import Stepper
stepper = Stepper(options=options, grid=(nx,))

from MPyDATA.solver import Solver
solver = Solver(stepper=stepper, advectee=advectee, advector=advector)

Initial state

In [ ]:
# plt.imshow(h_new, cmap='hot', interpolation='nearest')
axes = plt.gca()
plt.ylim(-.2, 1.5)

# plt.step([i for i in range(nx)], h_new, '-')
plt.plot(h_new, '-')
plt.show()

Main loop

In [ ]:
def MPDATA(advector, advectee):
    solver.curr.get()[:] = advectee
    solver.GC_phys.get_component(0)[:] = advector
    solver.advance(nt=1)
    return solver.curr.get() 

In [ ]:
g = 9.8 # m/s^2
for i in range(50):
    u_mid = extrapolate_in_time(interpolate_in_space(uh_new, h_new), interpolate_in_space(uh_old, h_old))
    
    #RHS = 0, mass conservation
    h_new[:] = MPDATA(u_mid, h_old)
    
    # -h * grad(h)   
    rhs_new = -g *  h_new * grad(h_new, dx=dx)

    #momentum conservation
    uh_new[:] = MPDATA(u_mid, uh_old + .5 * dt * rhs_old) + .5 * dt * rhs_new
    

    plt.ylim(-.5, 1.5)
    plt.plot(h_new, '-')
    plt.show()
#     plt.savefig(str(i) + '.png', format="png")
    
    # replace variables
    h_old, h_new = h_new, h_old
    u_old, u_new = u_new, u_old
    rhs_old, rhs_new = rhs_new, rhs_old
    uh_old, uh_new = uh_new, uh_old

    